# Data visualization for dataset hihgD

In [2]:
import numpy as np
import pandas as pd
import matplotlib as mlp
import matplotlib.animation as animation
import matplotlib.pyplot as plt
import os
import seaborn as sns

from copy import copy
from IPython.display import HTML
from matplotlib.patches import Rectangle, Ellipse
from pprint import pprint
from rich import print
from sys import platform
from time import time

from read_csv import read_track_csv, read_static_info, read_meta_info

sns.set()
sns.set_style("whitegrid")
sns.set_context("paper")
sns.color_palette("hls", 8)

def print_bl():
    print("\n")

%matplotlib inline
mlp.rcParams['animation.embed_limit'] = 2**128 #Increased animation size to save gifs if necessary


## Extracting frame data from the dataset.

The class SceneData contains the information of a single csv tuple (tracks, tracks metadata and video metadata). It handles data transformation from track information (in a per-vehicle trajectory basis) to parallel information, that is, which vehicles are present in the scene at each frame of the recording. It also handles distinction based on driving direction, so that vehicles in diferent roadways can be identified and field calculation is not impeded by taking into account vehicles driving in the other direction of the freeway.

It also contains a method to display or save a .gif reconstruction of the csv tuple. Beware, this reconstruction takes a long while to process, since the function that plots each frame of the animation is rather slow to execute - for the moment, no more work will be performed on it, but it is still considered to be a WIP.

The class will be updated with whatever additions are considered to extract the potential field that will be used to train the autoencoder further down the pipeline. Each major addition will have a corresponding subsection in this notebook. The goal is, then, to create a class that contains every transformation necessary, so that it can be then uploaded to the DGX server, which will be much faster in processing the information.

Code is self explanatory, except in some special dataframe accesses, which are suitably commented.

In [81]:
class SceneData:
    '''
    A class that represents an entire recording of the highD dataset.

    Attributes:
        dataset_location: a path to the directory in which the dataset is stored.
        dataset_index: index of the recording that is to be addressed
        sampling_period: time period between sampled frames. Needs to be a multiple of 40 ms
    '''
    def __init__(self, dataset_location = None, dataset_index = None, sampling_period = 40):
        self.dataset_index  = dataset_index
        self.dataset_location = dataset_location
        self.sampling_period = sampling_period
        self.df_location = dataset_location + str(dataset_index).zfill(2) + "_tracks.csv"
        self.static_info_location = dataset_location + str(dataset_index).zfill(2) + "_tracksMeta.csv"
        self.video_info_location = dataset_location + str(dataset_index).zfill(2) + "_recordingMeta.csv"
        
        self.bg_image = None
        self.bg_image_scaling_factor = None
        self.total_frame_number = 0
        self.longest_trajectory = 0

        self.frame_step = int(np.floor(self.sampling_period / 40))

        self.df = pd.read_csv(self.df_location)
        self.static_info = read_static_info(self.static_info_location)
        self.video_info = read_meta_info(self.video_info_location)
        self.frame_delay = 1000 / self.video_info["frameRate"]

        self.df_list = self.get_df_list()
        self.df_direction_list = self.get_df_direction_list()

        self.get_background_img(dataset_location + str(dataset_index).zfill(2) + "_highway.png")

    def get_background_img(self, path):
        '''
        Sets the background image for plotting and creating gifs.
            Parameters:
                path (str): path to the png file that contains the background image
            Returns:
                Nothing
        '''
        self.bg_image = plt.imread(path)
        self.bg_image_scaling_factor = np.max((np.max(self.bg_image.shape), self.longest_trajectory)) / np.min((np.max(self.bg_image.shape), self.longest_trajectory))  # Calculates the scaling factor between img size and trajectory length

    
    def get_df_list(self):
        '''
        Refactors information from track format to frame format.
            Parameters:
                None, inputs to the method are attributes of the class
            Returns:
                df_list: a list of dataframes - one per sampled frame - in which every vehicle in the scene is included.
        '''
        frame_groups = df.groupby((df.frame - 1) % frame_step == 0).get_group(1).groupby("frame")        
        self.total_frame_number = len(frame_groups)
        print("Frame step is {}".format(self.frame_step))
        print("Number of frames to be sampled is {}".format(self.total_frame_number))
        self.df_list = [None] * self.total_frame_number
        for idx, (_, group) in enumerate(frame_groups):
            frame_list[idx] = group
        return self.df_list

    def get_df_direction_list(self):
        '''
        Separates vehicles in frames according to the direction they are driving in.
            Parameters:
                None, inputs to the method are attributes of the class
            Returns:
                df_direction_list: a list of tuples - one tuple per every frame, with two elements: eastbound and westbound vehicles dataframes -.
        '''
        eastbound_vehicles = []  # contains the vehicle_id of every vehicle driving East
        westbound_vehicles = []  # contains the vehicle_id of every vehicle driving West
        for vehicle_id in self.static_info.keys():
            self.longest_trajectory = (
                self.static_info[vehicle_id]["traveledDistance"]
                if self.longest_trajectory
                < self.static_info[vehicle_id]["traveledDistance"]
                else self.longest_trajectory
            )  # get longest trajectory in dataset to calculate img scale factor
            if self.static_info[vehicle_id]["drivingDirection"] == 2.0:
                eastbound_vehicles.append(vehicle_id)
            else:
                westbound_vehicles.append(vehicle_id)

        self.df_direction_list = [None] * self.total_frame_number  # For every frame, save two dataframes - one for eastbound vehicles and one for westbound vehicles
        dummy_east, dummy_west = None, None
        for idx in range(len(self.df_list)):
            dummy_east = self.df_list[idx][
                [ident in eastbound_vehicles for ident in self.df_list[idx].id]
            ]  # eastbound vehicles
            dummy_west = self.df_list[idx][
                [ident in westbound_vehicles for ident in self.df_list[idx].id]
            ]  # westbound vehicles
            self.df_direction_list[idx] = [dummy_east, dummy_west]
        return self.df_direction_list

    def get_vehicle_groups_from_frame(self, frame):
        '''
        Given a frame, it creates a list in which every element is a dataframe with an ego vehicle and all those vehicles that are less closer than a predetermined distance away.
        The length of the list is the number of vehicles that are present in the frame.

        TODO: dynamic distance threshold - set to 50m at the time.
            Parameters:
                Frame: a dataframe containing information of a specific frame - see get_df_list and get_df_direction_list for details
            Returns:
                group_df_list: a list in which every element contains a vehicle group in the dataframe form
        '''

        def get_distance_between_vehicles(self, t1, t2):
            '''
            Gets the distance between two vehicles.
            Notation: 
                (x, y) - coordinates of the top left corner of the bounding box of the vehicle.
                (w, h) - width and height of the bounding box.

                Parameters:
                    t1: Tuple-like in the form (x1, y1, w1, h1).
                    t2: Tuple-like in the form (x2, y2, w2, h2)
                Returns:
                    dist: distance between vehicles
            '''
            x1, y1, w1, h1 = t1
            x2, y2, w2, h2 = t2
            c1 = np.array([x1 + w1/2, y1 + h1/2])
            c2 = np.array([x2 + w2/2, y2 + h2/2])
            dist = np.linalg.norm(c1-c2)
            return dist

        group_df_list = [None]*len(frame) #for every vehicle in the frame we will save a list of dataframes in which a vehicle group is represented
        drop_index_list = []

        for i ,(index, vehicle_id) in enumerate(frame.iterrows()): #iterate through every vehicle in the frame - returns index of entry and pd.Series with entry
            t1 = (vehicle_id.x, vehicle_id.y, vehicle_id.width, vehicle_id.height) #get position data of ego vehicle
            new_frame = frame.drop([index], axis = 0) #drop ego vehicle to not compare against itself
            drop_index_list.clear() #clear drop index list - it is faster to allocate it outside the loop than to create it in every iteration
            for index_local,vehicle_id_local in new_frame.iterrows(): #compare the ego vehicle against every vehicle in frame
                t2 = (vehicle_id_local.x, vehicle_id_local.y, vehicle_id_local.width, vehicle_id_local.height) #get position data of other vehicles
                if get_distance_between_vehicles(t1, t2) > 50: #if other vehicle is more than 50m away
                    drop_index_list.append(index_local) #store vehicles that are further than 50 m away
            new_frame = frame.drop(drop_index_list, axis = 0) #drop vehicles from dataframe
            index_list = list(new_frame.index) #get indices that were not dropped in a list
            index_list.remove(index) #remove the ego vehicle
            index_list.insert(0, index) #insert the ego vehicle in first place
            new_frame = new_frame.loc[index_list] #reorder the dataframe
            group_df_list[i] = new_frame #store filtered dataframe in list of frame groups

        return group_df_list
    
    def get_vehicle_groups(self):
        '''
        Given a list of frames, it creates the vehicle groups associated to them by calling get_vehicle_groups_from_frame.

            Parameters:
                None, inputs to this method are attributes of the class
            Returns:
                vehicle groups: a list in which every element is another list. The elements of the inner list are dataframes in which the information of a vehicle group is stored
        '''
        self.vehicle_groups = [None] * 2 * self.total_frame_number #preallocate for speed
        #get vehicle groups of eastbound vehicles
        for i in range(self.total_frame_number):
            frame = self.df_direction_list[i][0]
            self.vehicle_groups[i] = self.get_vehicle_groups_from_frame(frame)
            frame = self.df_direction_list[i][1]
            self.vehicle_groups[i+self.total_frame_number] = self.get_vehicle_groups_from_frame(frame)
        return self.vehicle_groups
    
    def save_vehicle_groups(self):
        """
        Saves the vehicle groups that were extracted from frame information into a csv file in the location of the dataset.

            Parameters:
                None, inputs to this method are attributes of the class.
            Returns:
                None.
        """
        file_path = self.dataset_location + str(self.dataset_index).zfill(2) + "_groups.csv"
        if os.path.exists(file_path): #if the file already exists, we need to erase it to avoid duplicity
            print("File " + file_path + " already exists. Deleting...")
            os.remove(file_path)
        self.vehicle_groups_flat = [group for frame in self.vehicle_groups for group in frame] #flatten the vehicle groups list
        with open(file_path, 'a') as f: #write the dataframes on the corresponding file
            for df in self.vehicle_groups_flat:
                df.to_csv(f, index=True, index_label='Index')
                f.write("\n")
        
    
    def plot_frame(self, frame_number=None):
        '''
        Visualization function. Plots the bounding boxes of every vehicle in a determined frame.

            Parameters:
                Frame number: frame to plot. If none is given, chosen at random
            Returns:
                Nothing.
        '''
        if frame_number is None:
            frame_number = np.random.randint(0, self.total_frame_number)

        eastbound_df, westbound_df = self.df_direction_list[frame_number]
        fig, ax = plt.subplots()
        if self.bg_image is not None:
            ax.imshow(self.bg_image)

        for index, vehicle in eastbound_df.iterrows():
            rect = Rectangle(
                (
                    int(vehicle.x * self.bg_image_scaling_factor),
                    int(vehicle.y * self.bg_image_scaling_factor),
                ),
                int(vehicle.width * self.bg_image_scaling_factor),
                int(vehicle.height * self.bg_image_scaling_factor),
                linewidth=1,
                edgecolor="r",
                facecolor="none",
            )
            ax.add_patch(rect)
        for index, vehicle in westbound_df.iterrows():
            rect = Rectangle(
                (
                    int(vehicle.x * self.bg_image_scaling_factor), #x
                    int(vehicle.y * self.bg_image_scaling_factor), #y
                ),
                int(vehicle.width * self.bg_image_scaling_factor), #width
                int(vehicle.height * self.bg_image_scaling_factor), #height
                linewidth=1,
                edgecolor="g",
                facecolor="none",
            )
            ax.add_patch(rect)
        plt.show()

    def plot_frame_anim(self, frame_number, ax):
        '''
        Visualization function. Plots the bounding boxes of every vehicle in a determined frame. Specifically created for animating a .gif

            Parameters:
                Frame number: frame to plot. Provided by the FuncAnimation class in matplotlib.
                ax: axis in which to plot the information. 
            Returns:
                Nothing.
        '''
        ax.clear()
        ax.imshow(self.bg_image)
        eastbound_df, westbound_df = self.df_direction_list[frame_number]
        for index, vehicle in eastbound_df.iterrows():
            rect = Rectangle(
                (
                    int(vehicle.x * self.bg_image_scaling_factor),
                    int(vehicle.y * self.bg_image_scaling_factor),
                ),
                int(vehicle.width * self.bg_image_scaling_factor),
                int(vehicle.height * self.bg_image_scaling_factor),
                linewidth=1,
                edgecolor="r",
                facecolor="none",
            )
            ax.add_patch(rect)
        for index, vehicle in westbound_df.iterrows():
            rect = Rectangle(
                (
                    int(vehicle.x * self.bg_image_scaling_factor),
                    int(vehicle.y * self.bg_image_scaling_factor),
                ),
                int(vehicle.width * self.bg_image_scaling_factor),
                int(vehicle.height * self.bg_image_scaling_factor),
                linewidth=1,
                edgecolor="g",
                facecolor="none",
            )
            ax.add_patch(rect)

    def plot_group(self, group_number = None):
        '''
        Visualization function. Plots the bounding boxes of every vehicle in a determined frame.

            Parameters:
                Group number: group to plot. If none is given, chosen at random
            Returns:
                Nothing.
        '''
        if group_number is None:
            group_number = np.random.randint(0, 2*self.total_frame_number)
        group_df = self.vehicle_groups[group_number]

        fig, ax = plt.subplots()
        if self.bg_image is not None:
            ax.imshow(self.bg_image)

        for index, vehicle in group_df.iterrows():
            rect = Rectangle(
                (
                    int(vehicle.x * self.bg_image_scaling_factor),
                    int(vehicle.y * self.bg_image_scaling_factor),
                ),
                int(vehicle.width * self.bg_image_scaling_factor),
                int(vehicle.height * self.bg_image_scaling_factor),
                linewidth=1,
                edgecolor="b",
                facecolor="none",
            )
            ax.add_patch(rect)
        plt.show()

    def create_gif(self, save_animation=False, frames=None, speed_up_factor=1):
        fig, ax = plt.subplots()
        fig.set_tight_layout(True)
        ax.grid(False)
        ax.set_xticks([])
        ax.set_yticks([])
        ax.imshow(self.bg_image)
        self.anim = animation.FuncAnimation(
            fig=fig,
            func=self.plot_frame_anim,
            fargs=(ax,),
            frames=frames if frames is not None else self.total_frame_number,
            interval=self.frame_delay / speed_up_factor,
            blit=False,
        )
        # HTML(anim.to_html5_video())

        if save_animation:
            writer = animation.PillowWriter(
                fps=scene_data.video_info["frameRate"] * speed_up_factor,
                metadata=dict(artist="Me"),
                bitrate=900,
            )
            anim.save("test.gif", writer=writer)
        # plt.show()
        return self.anim


## Class Instance and tests

Test of the implementation of dataframe collection and separation.

In [79]:
if platform == 'darwin':
    dataset_location = "/Users/lmiguelmartinez/Tesis/datasets/highD/data/"
else:
    dataset_location = "/home/lmmartinez/Tesis/datasets/highD/data/"
dataset_index = 1 #between 1 and 60

In [82]:
start = time()
scene_data = SceneData(dataset_location=dataset_location, dataset_index=dataset_index, sampling_period=1000)
end = time()
print("Time elapsed is:", end - start)
#anim = show_gif(scene_data=scene_data, frames = 500, speed_up_factor = 100)
#HTML(anim.to_jshtml())
#show_gif(scene_data=scene_data)

Frame step is 25

Number of frames to be sampled is 902

AttributeError: 'NoneType' object has no attribute 'id'

In [31]:
print(len(scene_data.df_direction_list))

22539

Test of the implementation of vehicle group extraction.

In [24]:
start = time()
vehicle_groups = scene_data.get_vehicle_groups()
end = time()
print("Time elapsed is:", end - start)

NameError: name 'scene_data' is not defined

Test of the implementation of group saving.

In [15]:
scene_data.save_vehicle_groups()
print(len(scene_data.vehicle_groups_flat))

File /Users/lmiguelmartinez/Tesis/datasets/highD/data/01_groups.csv already exists. Deleting...

13955

## Group indentification

This subsection covers the identification of groups of vehicles. For every vehicle in every frame, separated by direction, a dataframe in which the ego vehicle, as well as every vehicle that is closer than a predetermined threshold is stored. The threshold can be set to be either a hard parameter, or to depend on the ego vehicle speed.

In [6]:
def get_distance_between_vehicles(t1, t2):
    x1, y1, w1, h1 = t1
    x2, y2, w2, h2 = t2
    c1 = np.array([x1 + w1/2, y1 + h1/2])
    c2 = np.array([x2 + w2/2, y2 + h2/2])
    dist = np.linalg.norm(c1-c2)
    return dist

The following cell contains the transformations that are performed in the first frame of the dataset. The information flow is described in the commentaries of the code.

In [7]:
frame = scene_data.df_direction_list[7][0]
group_df_list = [None]*len(frame) #for every vehicle in the frame we will save a list of dataframes in which a vehicle group is represented

drop_index_list = []
index_list = []
for i ,(index, vehicle_id) in enumerate(frame.iterrows()): #iterate through every vehicle in the frame - returns index of entry and pd.Series with entry
    t1 = (vehicle_id.x, vehicle_id.y, vehicle_id.width, vehicle_id.height) #get position data of ego vehicle
    new_frame = frame.drop([index], axis = 0) #drop ego vehicle to not compare against itself
    drop_index_list.clear() #clear drop index list - it is faster to allocate it outside the loop than to create it in every iteration
    index_list.clear()
    for index_local,vehicle_id_local in new_frame.iterrows(): #compare the ego vehicle against every vehicle in frame
        t2 = (vehicle_id_local.x, vehicle_id_local.y, vehicle_id_local.width, vehicle_id_local.height) #get position data of other vehicles
        if get_distance_between_vehicles(t1, t2) > 50: #if other vehicle is more than 50m away
            drop_index_list.append(index_local) #store vehicles that are further than 50 m away
    new_frame = frame.drop(drop_index_list, axis = 0) #drop vehicles from dataframe
    index_list = list(new_frame.index)
    index_list.remove(index)
    index_list.insert(0, index)
    new_frame = new_frame.loc[index_list]
    group_df_list[i] = new_frame #store filtered dataframe in list of frame groups

# for group_df in group_df_list: #sanity check
#     display(group_df)

Once the code does what is expected, it is transformed into a function. These functions will be coded into the SceneData class as methods.

In [56]:
def get_vehicle_groups_from_frame(frame):
    group_df_list = [None]*len(frame) #for every vehicle in the frame we will save a list of dataframes in which a vehicle group is represented
    drop_index_list = []

    for i ,(index, vehicle_id) in enumerate(frame.iterrows()): #iterate through every vehicle in the frame - returns index of entry and pd.Series with entry
        t1 = (vehicle_id.x, vehicle_id.y, vehicle_id.width, vehicle_id.height) #get position data of ego vehicle
        new_frame = frame.drop([index], axis = 0) #drop ego vehicle to not compare against itself
        drop_index_list.clear() #clear drop index list - it is faster to allocate it outside the loop than to create it in every iteration
        for index_local,vehicle_id_local in new_frame.iterrows(): #compare the ego vehicle against every vehicle in frame
            t2 = (vehicle_id_local.x, vehicle_id_local.y, vehicle_id_local.width, vehicle_id_local.height) #get position data of other vehicles
            if get_distance_between_vehicles(t1, t2) > 50: #if other vehicle is more than 50m away
                drop_index_list.append(index_local) #store vehicles that are further than 50 m away
        new_frame = frame.drop(drop_index_list, axis = 0) #drop vehicles from dataframe
        group_df_list[i] = new_frame #store filtered dataframe in list of frame groups

    return group_df_list

def get_vehicle_groups(scene_data, frames = None):
    if frames is None:
        frames = scene_data.total_frame_number
    vehicle_groups = [None] * 2 * frames
    #get vehicle groups of eastbound vehicles
    for i in range(frames):
        frame = scene_data.df_direction_list[i][0]
        vehicle_groups[i] = get_vehicle_groups_from_frame(frame)
        frame = scene_data.df_direction_list[i][1]
        vehicle_groups[i+frames] = get_vehicle_groups_from_frame(frame)
    return vehicle_groups

In [9]:
vehicle_groups = get_vehicle_groups(scene_data, frames = 500)
len(vehicle_groups)

Saving the information is going to prove complicated. Firstly, let us flatten the list of lists of dataframes, so that we have a unique list of dataframes. Then, we can save the dataframes into a csv file.

In [11]:
vehicle_groups_flat = [group for frame in vehicle_groups for group in frame]

Once the list is flattened, we can save the dataframes into a csv file, writing an empty line between them.

In [13]:
with open(dataset_location + '01_groups.csv', 'a') as f:
    for df in vehicle_groups_flat:
        df.to_csv(f)
        f.write("\n")

We can read the dataframes now from the csv file. However, the reading tool included in pandas only reads one dataframe, and fills the empty lines with garbage. Moreover, the index column is mistreated and saved with no name, so we need to deal with that as well.

In [7]:
read_df = pd.read_csv("/Users/lmiguelmartinez/Tesis/datasets/highD/groups_1000ms/01_groups.csv")
print(read_df.columns)
read_df.head(20)

Index(['Index', 'frame', 'id', 'x', 'y', 'width', 'height', 'xVelocity',
       'yVelocity', 'xAcceleration', 'yAcceleration', 'frontSightDistance',
       'backSightDistance', 'dhw', 'thw', 'ttc', 'precedingXVelocity',
       'precedingId', 'followingId', 'leftPrecedingId', 'leftAlongsideId',
       'leftFollowingId', 'rightPrecedingId', 'rightAlongsideId',
       'rightFollowingId', 'laneId'],
      dtype='object')

,Index,frame,id,x,y,width,height,xVelocity,yVelocity,xAcceleration,...,precedingXVelocity,precedingId,followingId,leftPrecedingId,leftAlongsideId,leftFollowingId,rightPrecedingId,rightAlongsideId,rightFollowingId,laneId
0,0,1,1,362.26,21.68,4.85,2.12,40.85,0.0,0.3,...,0.0,0,3,0,0,0,0,0,6,5
1,Index,frame,id,x,y,width,height,xVelocity,yVelocity,xAcceleration,...,precedingXVelocity,precedingId,followingId,leftPrecedingId,leftAlongsideId,leftFollowingId,rightPrecedingId,rightAlongsideId,rightFollowingId,laneId
2,163,1,3,182.08,21.64,3.94,1.92,35.69,-0.0,0.17,...,40.85,1,7,0,0,0,10,11,0,5
3,848,1,7,151.54,22.21,4.75,2.02,32.59,-0.27,0.28,...,35.69,3,12,0,0,0,11,0,0,5
4,1463,1,10,201.31,25.62,9.2,2.5,23.21,0.17,0.17,...,23.27,6,11,1,0,3,0,0,0,6
5,1695,1,11,181.5,25.49,4.14,1.92,25.07,0.09,-0.29,...,23.21,10,0,1,3,7,0,0,0,6
6,Index,frame,id,x,y,width,height,xVelocity,yVelocity,xAcceleration,...,precedingXVelocity,precedingId,followingId,leftPrecedingId,leftAlongsideId,leftFollowingId,rightPrecedingId,rightAlongsideId,rightFollowingId,laneId
7,663,1,6,241.89,25.66,11.82,2.5,23.27,-0.0,-0.05,...,0.0,0,10,1,0,3,0,0,0,6
8,1463,1,10,201.31,25.62,9.2,2.5,23.21,0.17,0.17,...,23.27,6,11,1,0,3,0,0,0,6
9,Index,frame,id,x,y,width,height,xVelocity,yVelocity,xAcceleration,...,precedingXVelocity,precedingId,followingId,leftPrecedingId,leftAlongsideId,leftFollowingId,rightPrecedingId,rightAlongsideId,rightFollowingId,laneId


In [8]:
new_read_df = read_df #make a copy so the read dataframe is not modified

In [12]:
# Find the indices where the NaN values are located
nan_indices = new_read_df.index[new_read_df['Index'] == "Index"].tolist()
# Initialize an empty list to store the smaller DataFrames
smaller_dfs = [None]*len(nan_indices)
# Split the DataFrame based on the NaN indices
start = 0
for index, end in enumerate(nan_indices):
    smaller_df = new_read_df.iloc[start:end]
    smaller_df = smaller_df.set_index('Index') #Set the index column as index
    smaller_dfs[index] = (smaller_df)
    start = end + 1  # Set the start index for the next iteration
smaller_df = new_read_df.iloc[start:]
smaller_dfs.append(smaller_df)

In [11]:
display(smaller_dfs[-1])

,Index,frame,id,x,y,width,height,xVelocity,yVelocity,xAcceleration,...,precedingXVelocity,precedingId,followingId,leftPrecedingId,leftAlongsideId,leftFollowingId,rightPrecedingId,rightAlongsideId,rightFollowingId,laneId
53918,348736,22526,1047,379.82,12.47,6.37,3.13,-35.36,0.15,-0.74,...,-34.11,1045,0,0,0,0,1046,0,0,3


## History 

In [49]:
dataset_location = '/home/lmmartinez/Tesis/datasets/highD/data/01_tracks.csv'
sampling_period = 1000

df = pd.read_csv(dataset_location)
frame_step = int(np.floor(sampling_period / 40))

In [75]:
frame_groups = df.groupby((df.frame - 1) % frame_step == 0).get_group(1).groupby("frame")

In [76]:
frame_list = [None] * len(frame_groups)
for idx, (frame_number, group) in enumerate(frame_groups):
    frame_list[idx] = group


In [77]:
frame_list[0]

,frame,id,x,y,width,height,xVelocity,yVelocity,xAcceleration,yAcceleration,...,precedingXVelocity,precedingId,followingId,leftPrecedingId,leftAlongsideId,leftFollowingId,rightPrecedingId,rightAlongsideId,rightFollowingId,laneId
0,1,1,362.26,21.68,4.85,2.12,40.85,0.00,0.30,0.00,...,0.00,0,3,0,0,0,0,0,6,5
33,1,2,162.75,9.39,4.24,1.92,-32.04,0.00,-0.26,-0.00,...,0.00,0,8,0,0,4,0,0,0,2
163,1,3,182.08,21.64,3.94,1.92,35.69,-0.00,0.17,-0.00,...,40.85,1,7,0,0,0,10,11,0,5
320,1,4,267.34,13.12,5.05,2.22,-42.72,0.09,-0.08,0.00,...,0.00,0,5,0,0,0,8,0,9,3
481,1,5,309.40,13.81,4.24,1.82,-42.50,-0.08,-0.40,0.00,...,-42.72,4,0,0,0,0,8,0,9,3
663,1,6,241.89,25.66,11.82,2.50,23.27,-0.00,-0.05,-0.00,...,0.00,0,10,1,0,3,0,0,0,6
848,1,7,151.54,22.21,4.75,2.02,32.59,-0.27,0.28,0.01,...,35.69,3,12,0,0,0,11,0,0,5
1043,1,8,239.05,8.85,5.76,2.43,-31.36,-0.01,-0.17,-0.01,...,-32.04,2,9,0,0,4,0,0,0,2
1240,1,9,317.48,9.44,4.85,2.02,-35.99,-0.09,-0.16,0.02,...,-31.36,8,0,5,0,0,0,0,0,2
1463,1,10,201.31,25.62,9.20,2.50,23.21,0.17,0.17,-0.02,...,23.27,6,11,1,0,3,0,0,0,6
